In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 5 Questions
- Predict if patient survives stroke or not.
- Predict if patient will have disability.
- What are the main factors contributing to death?
- How does our model's predicted probability of death at 14 days/6 months align with the doctors predictions?
- What are positive factors for stroke survival?

In [3]:
stroke_trials_df = pd.read_csv('Resources/V2_International_Stroke_Trials.csv', low_memory=False)
stroke_trials_df.head()

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,RSBP,RCONSC,RDEF1,...,DDEADC,FPLACE,FAP,FOAC,OCCODE,FDEADC,CNTRYNUM,EXPDD,EXPD6,EXPD14
0,M,69,Y,NaN,Y,Y,NaN,140,D,N,...,0.0,E,NaN,NaN,2,NaN,27,0.6980,0.2344,0.1054
1,M,76,Y,NaN,Y,N,NaN,150,F,Y,...,0.0,A,NaN,NaN,2,NaN,27,0.5389,0.1555,0.0421
2,F,71,N,NaN,Y,N,NaN,170,F,Y,...,0.0,A,NaN,NaN,2,NaN,27,0.5275,0.1009,0.0323
3,M,81,N,NaN,N,N,NaN,170,F,N,...,0.0,A,NaN,NaN,4,NaN,27,0.4021,0.1147,0.0244
4,M,78,N,NaN,N,N,NaN,170,F,Y,...,0.0,E,NaN,NaN,2,NaN,27,0.5600,0.1709,0.0441


In [4]:
# View all columns
print(stroke_trials_df.columns)

Index(['SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RHEP24', 'RSBP',
       'RCONSC', 'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7',
       'RDEF8', 'STYPE', 'RXASP', 'RXHEP', 'DASP14', 'DASPLT', 'DLH14',
       'DMH14', 'DHH14', 'ONDRUG', 'DSCH', 'DIVH', 'DAP', 'DOAC', 'DGORM',
       'DSTER', 'DCAA', 'DHAEMD', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DDIAGISC',
       'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE',
       'DALIVE', 'DPLACE', 'DDEAD', 'DDEADC', 'FPLACE', 'FAP', 'FOAC',
       'OCCODE', 'FDEADC', 'CNTRYNUM', 'EXPDD', 'EXPD6', 'EXPD14'],
      dtype='object')


In [5]:
# Change all lowercase letters into uppercase
stroke_trials_df = stroke_trials_df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
stroke_trials_df.head(10)

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,RSBP,RCONSC,RDEF1,...,DDEADC,FPLACE,FAP,FOAC,OCCODE,FDEADC,CNTRYNUM,EXPDD,EXPD6,EXPD14
0,M,69,Y,NaN,Y,Y,NaN,140,D,N,...,0.0,E,NaN,NaN,2,NaN,27,0.6980,0.2344,0.1054
1,M,76,Y,NaN,Y,N,NaN,150,F,Y,...,0.0,A,NaN,NaN,2,NaN,27,0.5389,0.1555,0.0421
2,F,71,N,NaN,Y,N,NaN,170,F,Y,...,0.0,A,NaN,NaN,2,NaN,27,0.5275,0.1009,0.0323
3,M,81,N,NaN,N,N,NaN,170,F,N,...,0.0,A,NaN,NaN,4,NaN,27,0.4021,0.1147,0.0244
4,M,78,N,NaN,N,N,NaN,170,F,Y,...,0.0,E,NaN,NaN,2,NaN,27,0.5600,0.1709,0.0441
5,M,54,N,NaN,Y,N,NaN,135,F,Y,...,0.0,A,NaN,NaN,3,NaN,27,0.3132,0.0471,0.0244
6,F,77,N,NaN,N,N,NaN,140,F,Y,...,0.0,A,NaN,NaN,3,NaN,27,0.7285,0.1873,0.0530
7,M,23,N,NaN,Y,N,NaN,120,F,N,...,0.0,A,NaN,NaN,3,NaN,27,0.1500,0.0038,0.0054
8,M,47,N,NaN,N,N,NaN,150,F,N,...,0.0,A,NaN,NaN,3,NaN,27,0.1334,0.0162,0.0102
9,M,81,Y,NaN,N,N,NaN,170,F,N,...,0.0,A,NaN,NaN,2,NaN,27,0.4964,0.1476,0.0337


In [6]:
stroke_trials_df.describe(include='all')

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,RSBP,RCONSC,RDEF1,...,DDEADC,FPLACE,FAP,FOAC,OCCODE,FDEADC,CNTRYNUM,EXPDD,EXPD6,EXPD14
count,19435,19435.000000,19435,18451,19435,19435,19091,19435.000000,19435,19435,...,2334.000000,14933,14867,14861,19435.000000,4364.000000,19435.000000,19435.000000,19435.000000,19435.000000
unique,2,NaN,2,2,2,2,2,NaN,3,3,...,NaN,6,3,3,NaN,NaN,NaN,NaN,NaN,NaN
top,M,NaN,N,N,Y,N,N,NaN,F,Y,...,NaN,A,Y,N,NaN,NaN,NaN,NaN,NaN,NaN
freq,10407,NaN,13750,15282,13024,13020,18655,NaN,14921,14099,...,NaN,11607,10516,12323,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,71.715410,NaN,NaN,NaN,NaN,NaN,160.159197,NaN,NaN,...,2.404456,NaN,NaN,NaN,2.328891,3.441797,20.965629,0.629215,0.229676,0.094167
std,NaN,11.619714,NaN,NaN,NaN,NaN,NaN,27.610382,NaN,NaN,...,2.172007,NaN,NaN,NaN,1.068117,2.382524,8.439510,0.230564,0.194553,0.104473
min,NaN,16.000000,NaN,NaN,NaN,NaN,NaN,70.000000,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,1.000000,0.089100,0.003500,0.005000
25%,NaN,65.000000,NaN,NaN,NaN,NaN,NaN,140.000000,NaN,NaN,...,1.000000,NaN,NaN,NaN,2.000000,1.000000,14.000000,0.439100,0.084600,0.028500
50%,NaN,73.000000,NaN,NaN,NaN,NaN,NaN,160.000000,NaN,NaN,...,1.000000,NaN,NaN,NaN,2.000000,4.000000,25.000000,0.637800,0.160100,0.047100
75%,NaN,80.000000,NaN,NaN,NaN,NaN,NaN,180.000000,NaN,NaN,...,4.000000,NaN,NaN,NaN,3.000000,5.000000,27.000000,0.836750,0.319200,0.113100


# Address NaN Values

In [7]:
# Find the number of na values in each column
for col in stroke_trials_df.columns:
    if stroke_trials_df[col].isna().sum() > 0:
        print(f'{col}: {stroke_trials_df[col].isna().sum()}')

RATRIAL: 984
RHEP24: 344
DASP14: 22
DASPLT: 147
DLH14: 22
DMH14: 1006
DHH14: 18451
ONDRUG: 1
DSCH: 305
DIVH: 305
DAP: 18
DOAC: 18
DGORM: 23
DSTER: 28
DCAA: 29
DHAEMD: 28
DCAREND: 1005
DTHROMB: 315
DMAJNCH: 14
DDIAGISC: 23
DDIAGHA: 26
DDIAGUN: 23
DNOSTRK: 26
DRSISC: 18
DRSH: 15
DRSUNK: 1007
DPE: 14
DALIVE: 28
DPLACE: 9709
DDEAD: 19
DDEADC: 17101
FPLACE: 4502
FAP: 4568
FOAC: 4574
FDEADC: 15071


In [8]:
# Fill all blank columns, that use C, into C
C_columns = ['RATRIAL', 'RHEP24']
for col in C_columns:
    stroke_trials_df[col] = stroke_trials_df[col].fillna('C')

In [9]:
# Fill all blank columns, that use u, into U
U_columns = ['DASP14', 'DASPLT', 'DLH14', 'DMH14', 'DHH14', 'DSCH', 'DIVH', 'DAP', 'DOAC', 'DGORM', 'DSTER',
             'DCAA', 'DHAEMD', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH',
             'DRSUNK', 'DPE', 'DALIVE', 'DPLACE', 'DDEAD', 'FPLACE', 'FAP', "FOAC"]
for col in U_columns:
    stroke_trials_df[col] = stroke_trials_df[col].fillna('U')

In [10]:
# Fill all blank columns, that are numerical, into 0
numeric_columns = ['DDEADC', 'ONDRUG', 'FDEADC']
for col in numeric_columns:
    stroke_trials_df[col] = stroke_trials_df[col].fillna(0)

In [11]:
# Find the number of remaining na values in each column
for col in stroke_trials_df.columns:
    if stroke_trials_df[col].isna().sum() > 0:
        print(f'{col}: {stroke_trials_df[col].isna().sum()}')

In [12]:
# Drop all rows with na values
stroke_trials_dropna_df = stroke_trials_df.dropna()
stroke_trials_dropna_df.isna().sum().sum()

0

## Filter Out Heparin Patients
(outdated drug)

In [13]:
# Filter out rows that have 'Y' in the Heparin columns
heparin_columns = ['RHEP24', 'RXHEP', 'DLH14', 'DMH14', 'DHH14', 'DSCH', 'DIVH']
for col in heparin_columns:
    stroke_trials_dropna_df = stroke_trials_dropna_df[stroke_trials_dropna_df[col] != 'Y']
    stroke_trials_dropna_df = stroke_trials_dropna_df.drop(columns=[col])

stroke_trials_remove_hep_df = stroke_trials_dropna_df.copy()
stroke_trials_remove_hep_df.describe(include='all')

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RSBP,RCONSC,RDEF1,RDEF2,...,DDEADC,FPLACE,FAP,FOAC,OCCODE,FDEADC,CNTRYNUM,EXPDD,EXPD6,EXPD14
count,9773,9773.000000,9773,9773,9773,9773,9773.000000,9773,9773,9773,...,9773.000000,9773,9773,9773,9773.000000,9773.000000,9773.000000,9773.000000,9773.000000,9773.000000
unique,2,NaN,2,3,2,2,NaN,3,3,3,...,NaN,6,3,3,NaN,NaN,NaN,NaN,NaN,NaN
top,M,NaN,N,N,Y,N,NaN,F,Y,Y,...,NaN,A,Y,N,NaN,NaN,NaN,NaN,NaN,NaN
freq,5205,NaN,6904,7642,6286,6655,NaN,7506,7042,8342,...,NaN,5764,5196,6293,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,71.826358,NaN,NaN,NaN,NaN,160.236877,NaN,NaN,NaN,...,0.283332,NaN,NaN,NaN,2.323442,0.763328,21.285583,0.631200,0.231756,0.094968
std,NaN,11.666301,NaN,NaN,NaN,NaN,27.929060,NaN,NaN,NaN,...,1.061092,NaN,NaN,NaN,1.071941,1.804040,8.340225,0.231622,0.195793,0.105152
min,NaN,19.000000,NaN,NaN,NaN,NaN,71.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,1.000000,0.089100,0.004000,0.005100
25%,NaN,65.000000,NaN,NaN,NaN,NaN,140.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,2.000000,0.000000,14.000000,0.439100,0.085100,0.028400
50%,NaN,74.000000,NaN,NaN,NaN,NaN,160.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,2.000000,0.000000,25.000000,0.642200,0.161900,0.047600
75%,NaN,80.000000,NaN,NaN,NaN,NaN,180.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,3.000000,0.000000,27.000000,0.840100,0.322700,0.115000


## Evaluate Potential Target Variables

In [14]:
# Create dataFrames of target variables
target_14_days_df = stroke_trials_remove_hep_df[['DALIVE', 'DPLACE', 'DDEAD', 'DDEADC']].copy()
target_6_months_df = stroke_trials_remove_hep_df[['FPLACE', 'OCCODE', 'FDEADC']].copy()
display(target_14_days_df.head())
display(target_6_months_df.head())

,DALIVE,DPLACE,DDEAD,DDEADC
0,N,U,N,0.0
2,Y,U,N,0.0
5,Y,U,N,0.0
6,N,U,N,0.0
9,Y,U,N,0.0


,FPLACE,OCCODE,FDEADC
0,E,2,0.0
2,A,2,0.0
5,A,3,0.0
6,A,3,0.0
9,A,2,0.0


In [15]:
# See the value counts of each column in the target_14_days_df
for col in target_14_days_df.columns:
    print(f"{target_14_days_df[col].value_counts()}")

DALIVE
Y    5224
N    4524
U      25
Name: count, dtype: int64
DPLACE
U    4848
A    3573
E     801
D     271
C     141
B     139
Name: count, dtype: int64
DDEAD
N    8711
Y    1041
U      21
Name: count, dtype: int64
DDEADC
0.0    8733
1.0     521
4.0     194
2.0     113
5.0      77
6.0      45
7.0      44
8.0      29
3.0      17
Name: count, dtype: int64


- Drop rows with 'U' in DALIVE and DDEAD columns (only small number of rows)
- Decide what to do with DPLACE and DDEADC columns (drop DPLACE column, drop DDEADC column, drop 'U' rows in DPLACE, drop '0' rows in DDEADC, or a combination of those) - Going to drop both columns for now. 

In [16]:
# Drop rows that have 'U' in the DALIVE and DDEAD columns
stroke_trials_remove_hep_df = stroke_trials_remove_hep_df[(stroke_trials_remove_hep_df['DALIVE'] != 'U') & (stroke_trials_remove_hep_df['DDEAD'] != 'U')]
stroke_trials_remove_hep_df.describe(include='all')

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RSBP,RCONSC,RDEF1,RDEF2,...,DDEADC,FPLACE,FAP,FOAC,OCCODE,FDEADC,CNTRYNUM,EXPDD,EXPD6,EXPD14
count,9746,9746.000000,9746,9746,9746,9746,9746.000000,9746,9746,9746,...,9746.000000,9746,9746,9746,9746.000000,9746.000000,9746.000000,9746.000000,9746.000000,9746.000000
unique,2,NaN,2,3,2,2,NaN,3,3,3,...,NaN,6,3,3,NaN,NaN,NaN,NaN,NaN,NaN
top,M,NaN,N,N,Y,N,NaN,F,Y,Y,...,NaN,A,Y,N,NaN,NaN,NaN,NaN,NaN,NaN
freq,5191,NaN,6885,7621,6270,6638,NaN,7489,7024,8317,...,NaN,5757,5190,6287,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,71.818592,NaN,NaN,NaN,NaN,160.250564,NaN,NaN,NaN,...,0.282577,NaN,NaN,NaN,2.324236,0.760825,21.266981,0.631035,0.231545,0.094838
std,NaN,11.668252,NaN,NaN,NaN,NaN,27.929285,NaN,NaN,NaN,...,1.059530,NaN,NaN,NaN,1.067329,1.801928,8.341747,0.231549,0.195604,0.104976
min,NaN,19.000000,NaN,NaN,NaN,NaN,71.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,1.000000,0.089100,0.004000,0.005100
25%,NaN,65.000000,NaN,NaN,NaN,NaN,140.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,2.000000,0.000000,14.000000,0.439025,0.085100,0.028400
50%,NaN,74.000000,NaN,NaN,NaN,NaN,160.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,2.000000,0.000000,25.000000,0.642000,0.161650,0.047600
75%,NaN,80.000000,NaN,NaN,NaN,NaN,180.000000,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,3.000000,0.000000,27.000000,0.839575,0.322200,0.114875


In [17]:
# View the value counts of each column in the target_6_months_df
for col in target_6_months_df.columns:
    print(f"{target_6_months_df[col].value_counts()}")

FPLACE
A    5764
U    2316
D     654
B     448
C     352
E     239
Name: count, dtype: int64
OCCODE
2    3969
1    2148
3    1957
4    1620
0      49
9      30
Name: count, dtype: int64
FDEADC
0.0    7567
1.0     800
4.0     448
2.0     271
7.0     237
5.0     195
8.0     132
6.0      89
3.0      34
Name: count, dtype: int64


- Drop rows with '0' or '9' in OCCODE column (only small number of rows)
- Decide what to do with FPLACE and FDEADC columns (drop FPLACE column, drop FDEADC column, drop rows with 'U' in FPLACE, drop rows with '0' in FDEADC, or combination of those)

In [18]:
# Drop rows with '0' or '9' in the OCCODE column (unknown values)
stroke_trials_remove_hep_df = stroke_trials_remove_hep_df[(stroke_trials_remove_hep_df['OCCODE'] != 0) & (stroke_trials_remove_hep_df['OCCODE'] != 9)]
stroke_trials_remove_hep_df['OCCODE'].nunique()

4

In [19]:
# View the value counts of each column to see if there are any columns with a low count of unknown values that we can drop
for col in stroke_trials_remove_hep_df.columns:
    print(f"{col}: {stroke_trials_remove_hep_df[col].value_counts()}")

SEX: SEX
M    5151
F    4520
Name: count, dtype: int64
AGE: AGE
74    386
80    378
81    367
75    363
76    358
     ... 
26      2
19      1
99      1
20      1
21      1
Name: count, Length: 79, dtype: int64
RSLEEP: RSLEEP
N    6828
Y    2843
Name: count, dtype: int64
RATRIAL: RATRIAL
N    7554
Y    1571
C     546
Name: count, dtype: int64
RCT: RCT
Y    6215
N    3456
Name: count, dtype: int64
RVISINF: RVISINF
N    6586
Y    3085
Name: count, dtype: int64
RSBP: RSBP
160    1212
140    1158
150    1143
170     919
180     901
       ... 
226       1
217       1
295       1
238       1
223       1
Name: count, Length: 145, dtype: int64
RCONSC: RCONSC
F    7434
D    2113
U     124
Name: count, dtype: int64
RDEF1: RDEF1
Y    6961
N    2569
C     141
Name: count, dtype: int64
RDEF2: RDEF2
Y    8246
N    1356
C      69
Name: count, dtype: int64
RDEF3: RDEF3
Y    7260
N    2270
C     141
Name: count, dtype: int64
RDEF4: RDEF4
N    5080
Y    4280
C     311
Name: count, dtype: int64
RDEF5: 

In [154]:
# View the value counts of the FAP and FOAC columns for OCCODE = 1 (Death)
oc = stroke_trials_remove_hep_df[stroke_trials_remove_hep_df['OCCODE'] == 1]
oc[['FAP', 'FOAC']].value_counts()

FAP  FOAC
U    U       2135
Name: count, dtype: int64

### DataFrame representing patients Dead or Alive after 6 months

In [155]:
# Replace 2, 3 and 4 with 0 in OCCODE column
dead_or_alive_df = stroke_trials_remove_hep_df.copy()
dead_or_alive_df['OCCODE'] = dead_or_alive_df['OCCODE'].replace({1:0, 2:1, 3:1, 4:1})
dead_or_alive_df['OCCODE'].value_counts()

OCCODE
1    7536
0    2135
Name: count, dtype: int64

In [156]:
# View all the columns
stroke_trials_remove_hep_df.columns

Index(['SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RSBP', 'RCONSC',
       'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8',
       'STYPE', 'RXASP', 'DASP14', 'DASPLT', 'ONDRUG', 'DAP', 'DOAC', 'DGORM',
       'DSTER', 'DCAA', 'DHAEMD', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DDIAGISC',
       'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE',
       'DALIVE', 'DPLACE', 'DDEAD', 'DDEADC', 'FPLACE', 'FAP', 'FOAC',
       'OCCODE', 'FDEADC', 'CNTRYNUM', 'EXPDD', 'EXPD6', 'EXPD14'],
      dtype='object')

Potential Later Options:
- Create fourteen_day_df with DDEAD, DALIVE, DDEADC, or DPLACE as the y
- Replace OCCODE with FPLACE or FDEADC as the y
- Condensing columns into 1 (RDEF1-8 or DRSISC+DRSH+DRSUNK)
- Binning Age column

In [157]:
# Get important columns for the six month dead/alive model
six_month_dead_alive_df = dead_or_alive_df[['SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RSBP', 'RCONSC',
       'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8',
       'STYPE', 'RXASP', 'DASP14', 'DASPLT', 'ONDRUG', 'DAP', 'DOAC', 'DGORM',
       'DSTER', 'DCAA', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DCAREND', 'DHAEMD', 'DTHROMB',
       'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE', 'DALIVE', 'CNTRYNUM', 'OCCODE']]
six_month_dead_alive_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9671 entries, 0 to 19433
Data columns (total 41 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEX       9671 non-null   object 
 1   AGE       9671 non-null   int64  
 2   RSLEEP    9671 non-null   object 
 3   RATRIAL   9671 non-null   object 
 4   RCT       9671 non-null   object 
 5   RVISINF   9671 non-null   object 
 6   RSBP      9671 non-null   int64  
 7   RCONSC    9671 non-null   object 
 8   RDEF1     9671 non-null   object 
 9   RDEF2     9671 non-null   object 
 10  RDEF3     9671 non-null   object 
 11  RDEF4     9671 non-null   object 
 12  RDEF5     9671 non-null   object 
 13  RDEF6     9671 non-null   object 
 14  RDEF7     9671 non-null   object 
 15  RDEF8     9671 non-null   object 
 16  STYPE     9671 non-null   object 
 17  RXASP     9671 non-null   object 
 18  DASP14    9671 non-null   object 
 19  DASPLT    9671 non-null   object 
 20  ONDRUG    9671 non-null   float64


In [160]:
u_columns_to_drop = ['DASP14', 'ONDRUG', 'DAP', 'DOAC', 'DGORM', 'DSTER', 'DCAA', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DHAEMD', 'DTHROMB', 'DNOSTRK', 'DRSISC', 'DRSH', 'DPE']

for col in u_columns_to_drop:
    six_month_dead_alive_df = six_month_dead_alive_df[six_month_dead_alive_df[col]!= 'U']

six_month_dead_alive_df = six_month_dead_alive_df[six_month_dead_alive_df['ONDRUG'] != 0]

In [162]:
# Create csv files for the six month data
six_month_dead_alive_df.to_csv('Resources/six_month_dead_alive_data.csv', index=False)

### DataFrame representing patients Dead/Dependant or Alive after 6 months

In [163]:
# Replace 2's with a 1, and replace 3 & 4's with a 0 in OCCODE column
dead_or_depend_df = stroke_trials_remove_hep_df.copy()
dead_or_depend_df['OCCODE'] = dead_or_depend_df['OCCODE'].replace({2:1, 3:0, 4:0})
dead_or_depend_df['OCCODE'].value_counts()

OCCODE
1    6097
0    3574
Name: count, dtype: int64

In [164]:
# Get important columns for the six month dead/dependant or not model
six_month_dead_depend_df = dead_or_depend_df[['SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RSBP', 'RCONSC',
       'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8',
       'STYPE', 'RXASP', 'DASP14', 'DASPLT', 'ONDRUG', 'DAP', 'DOAC', 'DGORM',
       'DSTER', 'DCAA', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DCAREND', 'DHAEMD', 'DTHROMB',
       'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE', 'DALIVE', 'CNTRYNUM', 'OCCODE']]
six_month_dead_depend_df.head()

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RSBP,RCONSC,RDEF1,RDEF2,...,DHAEMD,DTHROMB,DNOSTRK,DRSISC,DRSH,DRSUNK,DPE,DALIVE,CNTRYNUM,OCCODE
0,M,69,Y,C,Y,Y,140,D,N,N,...,N,U,N,N,N,U,N,N,27,1
2,F,71,N,C,Y,N,170,F,Y,Y,...,N,U,N,N,N,U,N,Y,27,1
5,M,54,N,C,Y,N,135,F,Y,Y,...,N,U,N,N,N,U,N,Y,27,0
6,F,77,N,C,N,N,140,F,Y,Y,...,N,U,Y,N,N,U,N,N,27,0
9,M,81,Y,C,N,N,170,F,N,N,...,N,U,N,N,N,U,N,Y,27,1


In [165]:
# Create csv files for the six month dead/dependant (or not) data
six_month_dead_depend_df.to_csv('Resources/six_month_dead_depend_data.csv', index=False)

### DataFrame representing patients Dependant, Recovered, or Not-Recovered after 6 months

In [166]:
# Drop OCCODE with 1
alive_outcome_df = stroke_trials_remove_hep_df.copy()
alive_outcome_df = alive_outcome_df[alive_outcome_df['OCCODE'] != 1]
alive_outcome_df['OCCODE'].value_counts()

OCCODE
2    3962
3    1955
4    1619
Name: count, dtype: int64

In [167]:
# Get important columns for the six month dead/dependant or not model
six_month_alive_outcome_df = alive_outcome_df[['SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RSBP', 'RCONSC',
       'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8',
       'STYPE', 'RXASP', 'DASP14', 'DASPLT', 'ONDRUG', 'DAP', 'DOAC', 'DGORM',
       'DSTER', 'DCAA', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DCAREND', 'DHAEMD', 'DTHROMB',
       'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE', 'DALIVE', 'CNTRYNUM', 'FAP', 'FOAC', 'OCCODE']]
six_month_alive_outcome_df.head()

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RSBP,RCONSC,RDEF1,RDEF2,...,DNOSTRK,DRSISC,DRSH,DRSUNK,DPE,DALIVE,CNTRYNUM,FAP,FOAC,OCCODE
0,M,69,Y,C,Y,Y,140,D,N,N,...,N,N,N,U,N,N,27,U,U,2
2,F,71,N,C,Y,N,170,F,Y,Y,...,N,N,N,U,N,Y,27,U,U,2
5,M,54,N,C,Y,N,135,F,Y,Y,...,N,N,N,U,N,Y,27,U,U,3
6,F,77,N,C,N,N,140,F,Y,Y,...,Y,N,N,U,N,N,27,U,U,3
9,M,81,Y,C,N,N,170,F,N,N,...,N,N,N,U,N,Y,27,U,U,2


In [168]:
# Create csv files for the six month dead/dependant (or not) data
six_month_alive_outcome_df.to_csv('Resources/six_month_alive_outcome_data.csv', index=False)

### DataFrame representing doctor's predicted probability of death or death/dependent at 6 months

In [169]:
# Create dataFrame of doctor's predicted probabilities
doctor_pred_probs_df = stroke_trials_remove_hep_df[['EXPDD','EXPD6']].copy()
doctor_pred_probs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9671 entries, 0 to 19433
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EXPDD   9671 non-null   float64
 1   EXPD6   9671 non-null   float64
dtypes: float64(2)
memory usage: 226.7 KB


In [170]:
# round the values to nearest whole number
doctor_pred_probs_df = doctor_pred_probs_df.apply(lambda x: round(x,0))
doctor_pred_probs_df.head()

,EXPDD,EXPD6
0,1.0,0.0
2,1.0,0.0
5,0.0,0.0
6,1.0,0.0
9,0.0,0.0


In [171]:
# Create csv file for doctor's predicted probabilities
doctor_pred_probs_df.to_csv('Resources/doctor_6month_pred.csv', index=False)